# PyTorch Basics

## Init, helpers, utils, ...

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

In [2]:
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
from IPython.core.debugger import set_trace

# Tensors
tensors - the atoms of machine learning

## Tensors in numpy and pytorch

In [3]:
import numpy as np
from numpy.linalg import inv
from numpy.linalg import multi_dot as mdot

In [4]:
# numpy
np.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [5]:
# torch
torch.eye(3)

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

In [6]:
# numpy
X = np.random.random((5, 3))
X

array([[0.52704013, 0.04693532, 0.34383272],
       [0.96274192, 0.59859331, 0.95396384],
       [0.43564416, 0.40518508, 0.54073508],
       [0.59801273, 0.57840362, 0.53757271],
       [0.89522169, 0.23440955, 0.62413363]])

In [7]:
# pytorch
Y = torch.rand((5, 3))
Y

tensor([[0.1351, 0.1683, 0.8890],
        [0.9549, 0.8883, 0.5405],
        [0.9180, 0.1121, 0.3987],
        [0.7347, 0.0863, 0.9551],
        [0.2034, 0.4236, 0.8833]])

In [8]:
X.shape

(5, 3)

In [9]:
Y.shape

torch.Size([5, 3])

In [10]:
# numpy
X.T @ X

array([[2.55347024, 1.33328543, 2.21541599],
       [1.33328543, 0.91419041, 1.26350895],
       [2.21541599, 1.26350895, 1.99918958]])

In [11]:
# torch
Y.t() @ Y

tensor([[2.3539, 1.1234, 1.8834],
        [1.1234, 1.0169, 1.1310],
        [1.8834, 1.1310, 2.9337]])

In [12]:
# numpy
inv(X.T @ X)

array([[ 16.73349362,   9.67793816, -24.65989782],
       [  9.67793816,  14.24487761, -19.72758844],
       [-24.65989782, -19.72758844,  40.29528626]])

In [13]:
# torch
torch.inverse(Y.t() @ Y)

tensor([[ 1.1456, -0.7836, -0.4333],
        [-0.7836,  2.2577, -0.3673],
        [-0.4333, -0.3673,  0.7607]])

## More on PyTorch Tensors

Operations are also available as methods.

In [14]:
A = torch.eye(3)
A.add(1)

tensor([[2., 1., 1.],
        [1., 2., 1.],
        [1., 1., 2.]])

In [15]:
A

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

Any operation that mutates a tensor in-place has a `_` suffix.

In [16]:
A.add_(1)
A

tensor([[2., 1., 1.],
        [1., 2., 1.],
        [1., 1., 2.]])

## Indexing and broadcasting
It works as expected/like numpy:

In [17]:
A[0, 0]

tensor(2.)

In [18]:
A[0]

tensor([2., 1., 1.])

In [19]:
A[0:2]

tensor([[2., 1., 1.],
        [1., 2., 1.]])

In [20]:
A[:, 1:3]

tensor([[1., 1.],
        [2., 1.],
        [1., 2.]])

## Converting

In [21]:
A = torch.eye(3)
A

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

In [22]:
# torch --> numpy
B = A.numpy()
B

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

Note: torch and numpy can share the same memory / zero-copy

In [23]:
A.add_(.5)
A

tensor([[1.5000, 0.5000, 0.5000],
        [0.5000, 1.5000, 0.5000],
        [0.5000, 0.5000, 1.5000]])

In [24]:
B

array([[1.5, 0.5, 0.5],
       [0.5, 1.5, 0.5],
       [0.5, 0.5, 1.5]], dtype=float32)

In [25]:
# numpy --> torch
torch.from_numpy(np.eye(3))

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], dtype=torch.float64)

## Much more

In [26]:
[o for o  in dir(torch) if not o.startswith("_")]

['AVG',
 'AggregationType',
 'AnyType',
 'Argument',
 'ArgumentSpec',
 'Assert',
 'BFloat16Storage',
 'BFloat16Tensor',
 'BenchmarkConfig',
 'BenchmarkExecutionStats',
 'Block',
 'BoolStorage',
 'BoolTensor',
 'BoolType',
 'BufferDict',
 'ByteStorage',
 'ByteTensor',
 'CONV_BN_FUSION',
 'CallStack',
 'Capsule',
 'CharStorage',
 'CharTensor',
 'ClassType',
 'Code',
 'CompilationUnit',
 'CompleteArgumentSpec',
 'ComplexDoubleStorage',
 'ComplexFloatStorage',
 'ConcreteModuleType',
 'ConcreteModuleTypeBuilder',
 'CudaBFloat16StorageBase',
 'CudaBoolStorageBase',
 'CudaByteStorageBase',
 'CudaCharStorageBase',
 'CudaComplexDoubleStorageBase',
 'CudaComplexFloatStorageBase',
 'CudaDoubleStorageBase',
 'CudaFloatStorageBase',
 'CudaHalfStorageBase',
 'CudaIntStorageBase',
 'CudaLongStorageBase',
 'CudaShortStorageBase',
 'DeepCopyMemoTable',
 'DeviceObjType',
 'DictType',
 'DisableTorchFunction',
 'DoubleStorage',
 'DoubleTensor',
 'EnumType',
 'ErrorReport',
 'ExecutionPlan',
 'FUSE_ADD_REL

In [27]:
[o for o  in dir(A) if not o.startswith("_")]

['T',
 'abs',
 'abs_',
 'absolute',
 'absolute_',
 'acos',
 'acos_',
 'acosh',
 'acosh_',
 'add',
 'add_',
 'addbmm',
 'addbmm_',
 'addcdiv',
 'addcdiv_',
 'addcmul',
 'addcmul_',
 'addmm',
 'addmm_',
 'addmv',
 'addmv_',
 'addr',
 'addr_',
 'align_as',
 'align_to',
 'all',
 'allclose',
 'amax',
 'amin',
 'angle',
 'any',
 'apply_',
 'arccos',
 'arccos_',
 'arccosh',
 'arccosh_',
 'arcsin',
 'arcsin_',
 'arcsinh',
 'arcsinh_',
 'arctan',
 'arctan_',
 'arctanh',
 'arctanh_',
 'argmax',
 'argmin',
 'argsort',
 'as_strided',
 'as_strided_',
 'as_subclass',
 'asin',
 'asin_',
 'asinh',
 'asinh_',
 'atan',
 'atan2',
 'atan2_',
 'atan_',
 'atanh',
 'atanh_',
 'backward',
 'baddbmm',
 'baddbmm_',
 'bernoulli',
 'bernoulli_',
 'bfloat16',
 'bincount',
 'bitwise_and',
 'bitwise_and_',
 'bitwise_not',
 'bitwise_not_',
 'bitwise_or',
 'bitwise_or_',
 'bitwise_xor',
 'bitwise_xor_',
 'bmm',
 'bool',
 'byte',
 'cauchy_',
 'ceil',
 'ceil_',
 'char',
 'cholesky',
 'cholesky_inverse',
 'cholesky_solve

# But what about the GPU?
How do I use the GPU?

If you have a GPU make sure that the right pytorch is installed
(check https://pytorch.org/ for details).

In [28]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

/home/stefan/projects/pytorch_tutorial/.venv/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


device(type='cpu')

If you have a GPU you should get something like: 
`device(type='cuda', index=0)`

You can move data to the GPU by doing `.to(device)`.

In [29]:
data = torch.eye(3)
data.to(device)

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

Now the computation happens on the GPU.

In [30]:
res = data + data
res

tensor([[2., 0., 0.],
        [0., 2., 0.],
        [0., 0., 2.]])

In [31]:
res.device

device(type='cpu')

Note: before `v0.4` one had to use `.cuda()` and `.cpu()` to move stuff to and from the GPU.
This littered the code with many:
```python
if CUDA:
    model = model.cuda()
```

# Automatic differentiation with `autograd`
Prior to `v0.4` PyTorch used the class `Variable` to record gradients. You had to wrap `Tensor`s in `Variable`s.
`Variable`s behaved exactly like `Tensors`.

With `v0.4` `Tensor` can record gradients directly if you tell it do do so, e.g. `torch.ones(3, requires_grad=True)`.
There is no need for `Variable` anymore.
Many tutorials still use `Variable`, be aware!

Ref:
- https://pytorch.org/docs/stable/autograd.html
- https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html

You rarely use `torch.autograd` directly.
Pretty much everything is part or `torch.Tensor` now.
Simply add `requires_grad=True` to the tensors you want to calculate the gradients for.
`nn.Module` track gradients automatically.

In [32]:
from torch import autograd

In [33]:
x = torch.tensor(2.)
x

tensor(2.)

In [34]:
x = torch.tensor(2., requires_grad=True)
x

tensor(2., requires_grad=True)

In [35]:
print(x.requires_grad)

True


In [36]:
print(x.grad)

None


In [37]:
y = x ** 2

print("Grad of x:", x.grad)

Grad of x: None


In [38]:
y = x ** 2
y.backward()

print("Grad of x:", x.grad)

Grad of x: tensor(4.)


In [39]:
# What is going to happen here?
# x = torch.tensor(2.)
# x.backward()

In [40]:
# Don't record the gradient
# Useful for inference

params = torch.tensor(2., requires_grad=True)

with torch.no_grad():
    y = x * x
    print(x.grad_fn)

None


`nn.Module` and `nn.Parameter` keep track of gradients for you.

In [41]:
lin = nn.Linear(2, 1, bias=True)
lin.weight

Parameter containing:
tensor([[0.3237, 0.4990]], requires_grad=True)

In [42]:
type(lin.weight)

torch.nn.parameter.Parameter

In [43]:
isinstance(lin.weight, torch.FloatTensor)

True

# Exercise
- Do you remember the analytical solution to solve for the parameters of linear regression? Implement it.